In [2]:
import math
import pandas as pd
import numpy as np
from collections import Counter
from random import seed
import os
import traceback
import json
from enum import Enum


### Os projetos válidos para treinar o modelo vão ser os projetos que possuem 3 anos de informação antes da data de candidatura


In [1]:
def get_years(list_years, year_candidate):
    new_list = []
    for x in json.loads(list_years):
        if int(year_candidate) > int(x):
            new_list.append(x)
    return new_list

In [3]:
df_appi_facie = pd.read_csv('../DataFiles/APPIeFACIE-Resposta.csv', encoding="latin-1")

In [9]:
print("Nr projetos únicos: " + str(len(df_appi_facie['Nproj_anon'].unique())))

Nr projetos únicos: 2574


In [10]:
df_appi_facie.loc[(df_appi_facie['Tipo'] == "ENCPRJ") | (df_appi_facie['Tipo'] == "ENCINV")]

,Nproj_anon,Pedido,Tipo,Versão,Decisão,Formalização,Investimento contratado,Investimento apurado
0,15166,2,ENCPRJ,1.02s,2022-03-09,2021-12-16,"1,694,141.30 ","1,609,861.64 "
1,15166,2,ENCINV,1.02s,2020-03-06,2018-11-23,"1,694,141.30 ","1,609,861.64 "
4,15197,2,ENCPRJ,1.02s,2022-01-17,2021-12-16,"3,653,603.00 ","3,099,782.82 "
5,15197,2,ENCINV,1.02s,2019-11-14,2016-09-20,"3,653,603.00 ","3,099,782.82 "
18,15383,4,ENCINV,1.02s,2021-02-10,2018-01-03,"4,101,921.00 ","3,455,537.90 "
...,...,...,...,...,...,...,...,...
9265,1240193,3,ENCINV,1.02s,2022-01-28,2020-06-12,"503,500.00 ","490,398.01 "
9303,1240751,5,ENCINV,1.02s,2021-12-23,2020-03-11,"971,431.91 ","791,310.20 "
9341,1254577,7,ENCINV,1.02s,2021-12-29,2020-11-17,"2,048,571.55 ","2,023,977.57 "
9348,1254794,3,ENCINV,1.02s,2022-01-28,2020-03-03,"470,500.00 ","453,150.00 "


In [12]:
df_appi_facie.loc[(df_appi_facie['Tipo'] == "ENCPRJ") | (df_appi_facie['Tipo'] == "ENCINV"), ['Nproj_anon']].drop_duplicates(subset='Nproj_anon')

,Nproj_anon
0,15166
4,15197
18,15383
30,15662
46,15910
...,...
9265,1240193
9303,1240751
9341,1254577
9348,1254794


In [13]:
df_projects = df_appi_facie.groupby('Nproj_anon').agg({'Tipo':lambda x: ','.join(x)})
df_projects

,Tipo
Nproj_anon,
15166,"ENCPRJ,ENCINV,PPI,PPI"
15197,"ENCPRJ,ENCINV,PPI"
15259,"PPI,PPI,PPI,PPI,PPI,PPI"
15290,"PPI,PPI,PPI,PPI,PPI"
15383,"ENCINV,PPI,PPI,PPI,PPI"
...,...
5629204,PPI
5631095,PPI
5633327,PPI


In [15]:
df_projects_done_AND = df_projects[(df_projects['Tipo'].str.contains("ENCPRJ", case=False)) & (df_projects['Tipo'].str.contains("ENCINV", case=False))]
df_projects_done_AND

,Tipo
Nproj_anon,
15166,"ENCPRJ,ENCINV,PPI,PPI"
15197,"ENCPRJ,ENCINV,PPI"
16437,"ENCPRJ,ENCINV,PPI,PPI,PPI"
16809,"ENCPRJ,ENCINV,PPI"
16840,"ENCPRJ,ENCINV,PPI,PPI"
...,...
755787,"ENCPRJ,ENCINV,PPI,PPI,PPI"
758856,"ENCPRJ,ENCINV,PPI,PPI,PPI,PPI,PPI"
879260,"ENCPRJ,ENCINV,PPI,PPI,PPI"


In [17]:
df_projects_done_OR = df_projects[(df_projects['Tipo'].str.contains("ENCPRJ", case=False)) | (df_projects['Tipo'].str.contains("ENCINV", case=False))]
df_projects_done_OR

,Tipo
Nproj_anon,
15166,"ENCPRJ,ENCINV,PPI,PPI"
15197,"ENCPRJ,ENCINV,PPI"
15383,"ENCINV,PPI,PPI,PPI,PPI"
15662,"ENCINV,PPI,PPI,PPI,PPI,PPI,PPI,PPI"
15910,"ENCINV,PPI,PPI"
...,...
1240193,"ENCINV,PPI,PPI"
1240751,"ENCINV,PPI,PPI,PPI,PPI,PPI"
1254577,"ENCINV,PPI,PPI,PPI,PPI,PPI,PPI"
